In [1]:
import pandas as pd

In [2]:
url = 'http://d-infra.ier.hit-u.ac.jp/Japanese/ltes/LTES_08.xlsx'
xl = pd.ExcelFile(url)
sheet_name = '第2表'
df = xl.parse(sheet_name, header=None, index_col=None)

In [3]:
def create_cleaned_df(xl, sheet_name):
    df = xl.parse(sheet_name=sheet_name, header=None, index_col=None)
    df = drop_empty_columns_and_rows(df)
    rename_column_with_stat_id(df)
    df = get_rows_containing_data(df)
    return df


def drop_empty_columns_and_rows(df):
    return df.loc[df.notnull().sum(axis=1) > 2, df.notnull().sum() > 2]


def rename_column_with_stat_id(df):
    index = get_index_of_row_containing_stat_id(df)
    columns = df.loc[index].reset_index(drop=True).tolist()
    columns = rename_duplicates_in_list(columns)
    columns[0] = 'year_wst'
    df.columns = columns


def get_index_of_row_containing_stat_id(df):
    # All statistical IDs start with J.
    # Search from the second row
    index = df.index[df.iloc[:, 1].str.contains(r'^J', na=False)].tolist()[0]
    return index


def get_rows_with_digits(df):
    # Get rows with digts
    # Search from the first row
    rows_with_digits = df.iloc[:, 0].astype(str).str.isdigit().tolist()
    return rows_with_digits


def get_rows_containing_data(df):
    return df[get_rows_with_digits(df)]


def rename_duplicates_in_list(mylist):
    # Add underscores and progressive numbers to duplicate items in the list
    renamed_list = [str(v) + '_' + str(mylist[:i].count(v) + 1)
                    if mylist.count(v) >= 2 else v for i, v in enumerate(mylist)]
    return renamed_list


In [57]:
df1 = create_cleaned_df(xl, sheet_name)

In [58]:
df1.columns.values[-7] = 'year_wst_post'
    # CPI, 1789-1938, 1934-36 = 100, General, 'J0802__001'
    # CPI, 1789-1938, 1934-36 = 100, Urban Families: Food, 'J0802__005'
    # CPI, 1789-1938, 1934-36 = 100, Urban Families: Clothing, 'J0802__006'
    # CPI, 1946-1965, 1934-36 = 1, General, 'J0802__101'
    # CPI, 1951-1965, 1934-36 = 1, Food, 'J0802__102'
    # CPI, 1951-1965, 1934-36 = 1, Clothing, 'J0802__103'
df1_pre = df1[['year_wst',
                   'J0802__001', 'J0802__005', 'J0802__006']]
df1_post = df1[['year_wst_post',
                    'J0802__101', 'J0802__102',  'J0802__103']]
df1_post = df1_post.dropna()

In [59]:
df1_pre.year_wst = df1_pre.year_wst.astype(int).copy()

In [60]:
df1_pre.loc[:, 'year_wst'] = df1_pre.loc[:, 'year_wst'].astype(int).copy()

In [61]:
df1_post.loc[:, 'year_wst_post'] = df1_post.year_wst_post.astype(int)

In [62]:
    # Concatenate two data frames
df1 = df1_pre.merge(df1_post, how='outer',
                        left_on='year_wst', right_on='year_wst_post')

    # Create a column for the year
df1['year_wst'] = df1.year_wst.fillna(0) + df1.year_wst_post.fillna(0)

In [9]:
df1_post

,year_wst_post,J0802__101,J0802__102,J0802__103
22,1951,265.14,301.89,456.28
23,1952,277.79,315.33,384.12
24,1953,297.34,337.68,377.86
25,1954,314.89,362.24,378.13
26,1955,315.21,360.71,364.75
27,1956,317.55,360.68,366.15
28,1957,326.47,371.06,370.41
29,1958,328.2,371.84,365.36
30,1959,332.74,373.94,361.37
31,1960,342.91,385.02,367.83


In [63]:
columns_post = ['J0802__101', 'J0802__102',  'J0802__103']
adjustment = 100
for column in columns_post:
    df1_post[column] *= adjustment

In [64]:
columns_pair = {
    'J0802__001':'J0802__101',
    'J0802__005':'J0802__102',
    'J0802__006':'J0802__103'
}

In [65]:
columns_pre = ['J0802__001', 'J0802__005', 'J0802__006']

In [66]:
for column in columns_pre:
    df1[column]  = df1[column].fillna(0) + df1[columns_pair[column]].fillna(0) 

In [18]:
df1.to_csv('test.csv')

In [19]:
df1_post

,year_wst_post,J0802__101,J0802__102,J0802__103
22,1951,26514.0,30189.0,45628.0
23,1952,27779.0,31533.0,38412.0
24,1953,29734.0,33768.0,37786.0
25,1954,31489.0,36224.0,37813.0
26,1955,31521.0,36071.0,36475.0
27,1956,31755.0,36068.0,36615.0
28,1957,32647.0,37106.0,37041.0
29,1958,32820.0,37184.0,36536.0
30,1959,33274.0,37394.0,36137.0
31,1960,34291.0,38502.0,36783.0


In [20]:
df1

,year_wst,J0802__001,J0802__005,J0802__006,year_wst_post,J0802__101,J0802__102,J0802__103
0,1879.0,33.13,32.12,48.04,NaN,NaN,NaN,NaN
1,1880.0,37.95,38.44,56.37,NaN,NaN,NaN,NaN
2,1881.0,41.81,40.63,77.97,NaN,NaN,NaN,NaN
3,1882.0,38.91,36.21,66.56,NaN,NaN,NaN,NaN
4,1883.0,33.43,29.34,49.96,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
70,NaN,36003.00,40631.00,38030.00,1961.0,36003.0,40631.0,38030.0
71,NaN,38368.00,43485.00,40266.00,1962.0,38368.0,43485.0,40266.0
72,NaN,41173.00,47531.00,42756.00,1963.0,41173.0,47531.0,42756.0
73,NaN,42893.00,49438.00,43969.00,1964.0,42893.0,49438.0,43969.0


In [21]:
df1_merge = df1_pre.merge(df1_post, how = 'outer', left_on = 'year_wst', right_on = 'year_wst_post')

In [22]:
df1_merge

,year_wst,J0802__001,J0802__005,J0802__006,year_wst_post,J0802__101,J0802__102,J0802__103
0,1879.0,33.13,32.12,48.04,NaN,NaN,NaN,NaN
1,1880.0,37.95,38.44,56.37,NaN,NaN,NaN,NaN
2,1881.0,41.81,40.63,77.97,NaN,NaN,NaN,NaN
3,1882.0,38.91,36.21,66.56,NaN,NaN,NaN,NaN
4,1883.0,33.43,29.34,49.96,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,1961.0,36003.0,40631.0,38030.0
71,NaN,NaN,NaN,NaN,1962.0,38368.0,43485.0,40266.0
72,NaN,NaN,NaN,NaN,1963.0,41173.0,47531.0,42756.0
73,NaN,NaN,NaN,NaN,1964.0,42893.0,49438.0,43969.0


In [23]:
df1_merge.loc[:,'year_wst'] = df1_merge.year_wst.fillna(0) + df1_merge.year_wst_post.fillna(0)

In [24]:
df1_merge

,year_wst,J0802__001,J0802__005,J0802__006,year_wst_post,J0802__101,J0802__102,J0802__103
0,1879.0,33.13,32.12,48.04,NaN,NaN,NaN,NaN
1,1880.0,37.95,38.44,56.37,NaN,NaN,NaN,NaN
2,1881.0,41.81,40.63,77.97,NaN,NaN,NaN,NaN
3,1882.0,38.91,36.21,66.56,NaN,NaN,NaN,NaN
4,1883.0,33.43,29.34,49.96,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
70,1961.0,NaN,NaN,NaN,1961.0,36003.0,40631.0,38030.0
71,1962.0,NaN,NaN,NaN,1962.0,38368.0,43485.0,40266.0
72,1963.0,NaN,NaN,NaN,1963.0,41173.0,47531.0,42756.0
73,1964.0,NaN,NaN,NaN,1964.0,42893.0,49438.0,43969.0


In [25]:
sheet_name4 = '第1表'
df4 = create_cleaned_df(xl, sheet_name4)
    # Create a temporary data frame to get the linked index.
df4_temp = xl.parse(sheet_name4, header=None, index_col=None)

In [26]:
value_1955 = df4_temp.iloc[97,-1]
linked_index_1955 = df4_temp.iloc[109,-1]

In [27]:
value_1955,linked_index_1955

(82.8, 359.7)

In [28]:
adjustment_df4 = linked_index_1955 / value_1955 * 100

In [29]:
adjustment * value_1955

8280.0

In [30]:
df4['J0801__003'] = df4['J0801__003_1'].fillna(0) +  df4['J0801__003_2'].fillna(0) * adjustment_df4
df4 = df4[['year_wst', 'J0801__003']]

In [31]:
df4

,year_wst,J0801__003
21,1879,30.800000
22,1880,28.900000
23,1881,30.200000
24,1882,29.900000
25,1883,31.900000
...,...,...
102,1960,43268.260870
103,1961,45961.666667
104,1962,46396.086957
105,1963,47308.369565
